# MNIST

In [26]:
import sys, os
sys.path.append(os.pardir)

In [2]:
# jupyter notebookのホームディレクトリにファイルが有れば読める？分からん．
# 内容としては，gzファイルを解凍して，出てきた画像をnumpyで読めるように頑張ってくれている．
# 前処理で画素値の正規化や入力画像を1次元配列にする処理，ワンホットでラベルを作成する話．
# ファイルの内容をnp.array()にしたい時は，np.frombuffer()を使うと良いみたいですね．
# http://inatim.com/numpy-frombuffer/
from mnist import load_mnist

In [3]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

In [4]:
x_train.shape

(60000, 784)

### 試しに画像を表示してみる

In [10]:
import numpy as np
from PIL import Image
import pickle

In [6]:
def image_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

In [7]:
#　1次元にしてあるので，reshapeで28＊28の画像に作り変える必要がある．
image_show(x_train[0].reshape(28,28))

In [9]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

In [28]:
def init_network():
    with open("../sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
        
    return network

In [13]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [16]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    return exp_a / sum_exp_a

In [64]:
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3'],
    b1, b2, b3 = network['b1'], network['b2'], network['b3'],

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

In [18]:
x, t = get_data()

In [29]:
network = init_network()

In [65]:
accuracy_count = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    
    if p == t[i]:
        accuracy_count += 1

In [50]:
float(accuracy_count) / len(x)

0.9352

In [60]:
x.shape

(10000, 784)

In [72]:
W1, W2, W3 = network['W1'], network['W2'], network['W3']

In [73]:
W1.shape

(784, 50)

In [75]:
W2.shape

(50, 100)

In [71]:
W3.shape

(100, 10)

In [65]:
batch_size = 100
accuracy_count = 0
for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    
    if p == t[i]:
        accuracy_count += 1